### Look-up table from FAA DRS

In [1]:
import pandas as pd
import os

metadata_mapping = pd.read_excel(os.path.join(os.getcwd(),'FAA DRS','DRS Document Types Metadata Mapping.xlsx')) # from https://drs.faa.gov/help/helpdetails and https://drs.faa.gov/help/helpdetails
doc_types = pd.unique(metadata_mapping.loc[:, "Document Type Name in API request"])
doc_types

array(['ADFRAWD', 'ADNPRM', 'AC', 'AB', 'POLICY', 'CAM', 'CAR', 'CANIC',
       'ADFREAD', 'ELOS', 'CFRFRSFAR', 'NORSEE', 'NPRM', 'PMA', 'SAIB',
       'SCFINAL', 'SCPROPOSED', 'SFAR', 'STC', 'TSOI', 'TSO', 'FAR',
       'TCDSMODEL', 'UNAPPROVED_PARTS_NOTIFICATIONS', 'INFO', 'SAFO',
       'ORDER_8900.1', 'CLARIFY_POLICY', 'AFS-1_MEMORANDUMS', 'AT_JTA',
       'AIRCRAFT_STANDARDIZED_CURRICULUM', 'AIRCRAFT_MASTER_SCHEDULE',
       'OTHER_AWO', 'ALERTS', 'OTHER_AWARDS_INFORMATION_GUIDES',
       '8900.1_EDITORIAL_CORRECTIONS', 'OTHER_EFB_RESEARCH_REPORTS',
       'OTHER_EFB_CHECKLISTS', 'OTHER_FAA_90_DAY_SAFETY_REVIEW',
       'OTHER_CPD_6.03', 'AFS_FFS_UPDATEPUB', 'AFS_FFS_UPDATES', 'FOEB',
       'FSB_REPORTS', 'AFS_POLICY_DEV_MEMOS',
       'OTHER_FLIGHT_STANDARDS_ORM_WORKSHEETS', 'AFS_FOCUS_TEAMS',
       'GA_JTA', 'BULLETINS', 'OTHER_PS_HANDBOOKS',
       'OTHER_INFORMATION_GUIDES', 'PILOT_QUALIFICATION_CURRICULUM',
       'OTHER_INTERNATIONAL_PUBLICATIONS', 'OTHER_JOB_AIDS',
      

#### We just want to collect STCs

In [2]:
doc_types = ['STC'] 

### Just a test to collect STC list

In [2]:
import requests
import os
df_drs = None
doc_type = 'STC'

with open(os.path.join(os.getcwd(),'drs_api.txt'), "r") as f:
    x_api_key = f.read()

offset = 1250

url = "https://drs.faa.gov/api/drs/data-pull/"+doc_type+"?offset="+str(offset) # ?drs:offset ?drs:offset
headers = {'x-api-key':x_api_key}
response = requests.get(url, headers=headers)
print(url)

df_drs = pd.DataFrame.from_dict(response.json()['documents'])

df_drs.to_excel(os.path.join(os.getcwd(),'database','stc_collection_test_offset='+str(offset)+'.xlsx'),index=False)

In [4]:
df_drs.loc[:, :]

AttributeError: 'NoneType' object has no attribute 'loc'

### Collect full STC list from FAA DRS

In [8]:
import logging
import requests
import os
import datetime

df_drs = None
logging.basicConfig(filename='10_stc_collection.log', encoding='utf-8',format='%(asctime)s %(message)s', level=logging.INFO)

stc_path = os.path.join(os.getcwd(),'database','stc.xlsx')

with open(os.path.join(os.getcwd(),'drs_api.txt'), "r") as f:
    x_api_key = f.read()

for doc_type in doc_types:
    hasMoreItems = True
    offset = 0
    logging.info("FileType: "+doc_type)

    while hasMoreItems:
        try:
            url = "https://drs.faa.gov/api/drs/data-pull/"+doc_type+"?offset="+str(offset)
            headers = {'x-api-key':x_api_key}
            response = requests.get(url, headers=headers)

            if df_drs is None:
                df_drs = pd.DataFrame.from_dict(response.json()['documents'])
            else:
                df_drs = pd.concat([df_drs, pd.DataFrame.from_dict(response.json()['documents'])])
            
            hasMoreItems = response.json()['summary']['hasMoreItems'] and (offset<response.json()['summary']['totalItems'])
            logging.info("Passed with Offset: "+str(offset)+"/"+str(response.json()['summary']['totalItems'])+"; hasMoreItems="+str(response.json()['summary']['hasMoreItems'])+"; url: "+url)
            offset = offset + response.json()['summary']['count']
        except:
            logging.info("Failed with Offset: "+str(offset)+"; url: "+url)

        df_drs.to_excel(stc_path,index=False)

try:
    #os.rename(os.path.join(os.getcwd(),'stc.xlsx'), os.path.join(os.getcwd(),'data','stc','stc_'+datetime.datetime.now().strftime("%Y%m%d"-"%H%M%S")+'.xlsx'))
    os.rename(stc_path, 
              os.path.join(os.getcwd(),'database','stc_'+datetime.datetime.now().strftime("%Y%m%d - %H%M%S")+'.xlsx'))
except:
    pass

## Collecting PDFs from FAA DRS

#### A quick try to collect for one given STC

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import re
import glob
import os

dl_dir = os.path.join(os.getcwd(),'database','data','stc','pdf')

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : dl_dir}
chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(chrome_options=chrome_options)

#driver.get("https://drs.faa.gov/browse/excelExternalWindow/BFEFEB9191F15EB286258876006E8C75.0001?modalOpened=true")
#driver.get("https://drs.faa.gov/browse/excelExternalWindow/C467DBFA533DC75A85256CC2000E3B05.0001?modalOpened=true")
#collect_drs_pdf("https://drs.faa.gov/browse/excelExternalWindow/C467DBFA533DC75A85256CC2000E3B05.0001?modalOpened=true", driver)
stc_name_from_site = collect_drs_pdf("https://drs.faa.gov/browse/excelExternalWindow/4AED288A0E84EDD3862579640077B32A.0001?modalOpened=true", driver)

list_of_files = glob.glob(os.path.join(dl_dir,"*.pdf"))
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

if os.path.exists(os.path.join(dl_dir, stc_name_from_site)) and (stc_name_from_site in latest_file):
    os.rename(latest_file, os.path.join(dl_dir, "SB03032CH"+"__"+"Current"+"__"+"4AED288A0E84EDD3862579640077B32A"+".pdf"))

C:\Users\victor\AppData\Local\Temp\ipykernel_17668\2185806852.py:15: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


C:\Users\victor\Documents\DeepLearning\FAA NLP Project\database\data\stc\pdf\SB03032CH.pdf


#### Main function for DRS PDF collector 

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime
import os.path

def collect_drs_pdf(url, driver):
    driver.get(url)
    stc_name_from_site = "NO STC DOWNLOADED"
    dl_dir = os.path.join(os.getcwd(),'database','data','stc','pdf')

    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "[title*='Supplemental Type Certificates (STC)']")) 
        )  

        if len(driver.find_elements(By.ID, "printButton"))>0 and len(driver.find_elements(By.ID, "STC Number"))>0:
            stc_name_from_site = "no_pdf_stc_"+datetime.datetime.now().strftime("%H%M%S%f")+".pdf"
            with open(os.path.join(dl_dir,stc_name_from_site), 'w') as f:
                f.write(url)
        else:
            WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#download"))
            )
            time_counter = 0
            while driver.find_element(By.CSS_SELECTOR, "#loadingBar").get_attribute("class")!="hidden" and time_counter<60:
                time.sleep(0.5)
                time_counter = time_counter + 1
            driver.find_element(By.CSS_SELECTOR, "#download").click()

            stc_name_from_site = driver.find_element(By.CLASS_NAME, "doc-view").find_element(By.CLASS_NAME, "ng-star-inserted").text
            if stc_name_from_site[0:2]=="- ":
                stc_name_from_site = stc_name_from_site[2:]

            time_counter = 0
            while not os.path.exists(os.path.join(dl_dir, stc_name_from_site)) and time_counter<60:
                time.sleep(0.5)
                time_counter = time_counter + 1
            time.sleep(0.5)
    except TimeoutException as ex:
        pass
    except:
        pass        

    return stc_name_from_site

#### Collects all DRS PDF no optimization using the previously collected full STC list from FAA DRS

In [1]:
import pandas as pd
import os

df_stc = pd.read_excel(os.path.join(os.getcwd(),'database','stc.xlsx'))
print(df_stc.shape)
df_stc = df_stc.drop_duplicates()
print(df_stc.shape)

(77965, 26)
(77661, 26)


In [3]:
from selenium import webdriver
import os
import logging
import re
import glob
import time
regcomp = re.compile(" \(")

dl_dir = os.path.join(os.getcwd(),'database','data','stc','pdf')

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : dl_dir}
chrome_options.add_experimental_option('prefs', prefs)
logging.basicConfig(filename='drs_stc_collection.log', encoding='utf-8',format='%(asctime)s %(message)s', level=logging.INFO)
logging.info("DRS PDF collector no optimization")

driver = webdriver.Chrome(chrome_options=chrome_options)
for index, row in df_stc.sample(frac=1).iterrows():
    matched_stc = regcomp.search(row['drs:chronicleId'])
    if matched_stc is not None:
        stc_num = row['drs:chronicleId'][0:matched_stc.span()[0]]
    else:
        stc_num = row['drs:chronicleId']

    if not os.path.exists(os.path.join(dl_dir,row['drs:chronicleId']+"__"+row['drs:status']+"__"+row['documentGuid']+".pdf")):
        try:            
            stc_name_from_site = collect_drs_pdf(row['documentURL']+"?modalOpened=true", driver)

            list_of_files = glob.glob(os.path.join(dl_dir,"*.pdf"))
            latest_file = sorted(list_of_files, key=os.path.getctime, reverse=True)[:10]

            time.sleep(0.2)
            if (stc_name_from_site in [os.path.basename(val) for val in latest_file]):
                os.rename(os.path.join(dl_dir, stc_name_from_site), os.path.join(dl_dir, row['drs:chronicleId']+"__"+row['drs:status']+"__"+row['documentGuid']+".pdf"))
                
            logging.info("drs:chronicleId: "+row['drs:chroni  cleId']+" -> Pdf dl Pass")
        except:
            logging.info("drs:chronicleId: "+row['drs:chronicleId']+" -> Pdf dl Fail")

driver.quit()

C:\Users\victor\AppData\Local\Temp\ipykernel_13400\519162152.py:17: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [1]:
import os
"no_pdf_stc_211603973870.pdf" in ['no_pdf_stc_211603973870.pdf', 'no_pdf_stc_211601549633.pdf', 'ST02959AT.pdf', 'no_pdf_stc_211555126680.pdf', 'no_pdf_stc_211550821480.pdf', 'no_pdf_stc_211548432536.pdf', 'no_pdf_stc_211546012346.pdf', 'SA02571AT-D.pdf']

True

#### Remove files incorrectly downloaded

In [ ]:
import os
import glob

dl_dir = os.path.join(os.getcwd(),'database','data','stc')

for file in glob.glob(os.path.join(os.path.join(dl_dir,'pdf'),"*.pdf")):
    if "__" not in os.path.basename(file):
        try:
            os.rename(file, os.path.join(os.path.join(os.path.join(dl_dir,'pdf did not work')), os.path.basename(file)))
        except:
            os.remove(file)


#### Collects all DRS PDF multi thread optimization

In [ ]:
import pandas as pd
from multiprocessing.pool import ThreadPool as Pool
import os

path = os.path.join(os.getcwd(),'database','data','stc','pdf')
stcs = ["SR02762SE.pdf", "SR02650SE.pdf", "SR04501CH.pdf", "ST04442NY.pdf", "SB04185CH.pdf", "SR04557NY.pdf", "SR01813CH.pdf", "SH5977NM.pdf"]
urls = [val for val in stcs]

df = pd.DataFrame(stcs, columns=['stcs'])

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(collect_drs_pdf, [1, 2, 3]))

def sitesize(url):
    with requests.get(url) as u:
        return url, len(u.content)

pool = Pool(10)
for result in pool.imap_unordered(sitesize, sites):
    print(result)

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from functools import partial
from itertools import repeat
#from multiprocessing import Pool
from multiprocess import Pool
import os

urls = ["https://drs.faa.gov/browse/excelExternalWindow/d2b9293f-76c6-499d-8328-dffcda388dc5?modalOpened=true", 
        "https://drs.faa.gov/browse/excelExternalWindow/0D55897B0F4DADC885256CC100821507.0001?modalOpened=true", 
        "https://drs.faa.gov/browse/excelExternalWindow/5850CE209CB40D16862579E2006009B7.0001?modalOpened=true", 
        "https://drs.faa.gov/browse/excelExternalWindow/606dd138-75b4-4d49-96c6-e525c00a6730?modalOpened=true", 
        "https://drs.faa.gov/browse/excelExternalWindow/7b1708f9-c7c9-4020-8779-c64eb3e80e0b?modalOpened=true"]

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : os.path.join(os.getcwd(),'database','data','stc','pdf')}

chrome_options.add_experimental_option('prefs', prefs)
webdriver = webdriver.Chrome(chrome_options=chrome_options)

#collect_drs_pdf("https://drs.faa.gov/browse/excelExternalWindow/FAA000000000LEGALINTPR2013001PDF.0001?modalOpened=true", webdriver)

df = pd.DataFrame(urls, columns=['urls'])

with Pool(1) as p:
    p.map_async(collect_drs_pdf, [(urls[0], webdriver)])

'''
with Pool() as pool:
    pool.map(collect_drs_pdf,[urls[0], webdriver])

with Pool() as pool:
    pool.starmap(collect_drs_pdf,[(urls[0], webdriver)])
    #pool.starmap(collect_drs_pdf, list(zip(iter(urls), repeat(webdriver))))
    #pool.starmap(collect_drs_pdf, zip(urls, repeat(webdriver)))
'''

C:\Users\victor\AppData\Local\Temp\ipykernel_8144\4141224413.py:20: DeprecationWarning: use options instead of chrome_options
  webdriver = webdriver.Chrome(chrome_options=chrome_options)
